In [19]:
from collections import defaultdict, Counter
import pprint
from operator import itemgetter
import re,operator
import bz2, json
from collections import defaultdict
import cmath
WORD_LEN_COEFF = 1
THRESHOLD_COEFF = 0.5
DROP = 1.5
DROP_1 = 2
AFFIX_LEN = 1

In [20]:
svoc = bz2.BZ2File('voc.json.bz2', 'r').read().decode(encoding='utf-8')
voc = json.loads(svoc)
del svoc

In [21]:
words = list(voc.keys())
word_count = sum([voc[k] for k in voc])
average_word_len = sum([len(w)*voc[w] for w in words]) / word_count
len_search = int(average_word_len * WORD_LEN_COEFF) #это максимальная разрешенная длина аффикса
print("{} словоформ, {} словоупотреблений, средняя длина слова {} ".format(len(words), word_count, average_word_len))

386233 словоформ, 11859267 словоупотреблений, средняя длина слова 7.1053179762290535 


In [128]:
k=0
for word in words:
    if word.endswith('аларыгар'):
        k+=1
print(k)

418


In [22]:
# подсчет безусловных вероятностей букв
# trie, prob = build_trie_and_prob(voc)
prob = json.load(open("prob.json", encoding="utf-8"))
strie = bz2.BZ2File('trie.json.bz2', 'r').read().decode(encoding='utf-8')
trie = json.loads(strie)
del strie
print("Безусловные вероятности первых 10 букв:\n========================\n",
sorted([(letter,nv) for letter,nv in prob.items()], key=itemgetter(1), reverse=True)[:10])

Безусловные вероятности первых 10 букв:
 [('а', 0.13174638100795355), ('т', 0.07463523242460413), ('н', 0.07183703410321932), ('р', 0.06989005476760542), ('э', 0.06769593508904286), ('и', 0.06497202721408583), ('ы', 0.05657750345483212), ('л', 0.052736912856701096), ('у', 0.048080219156342266), ('о', 0.04801633649290444)]


In [ ]:
letters = list(prob.keys())
cond_prob = defaultdict(lambda: 0) #словарь для условных вероятностей
total = defaultdict(lambda: 0)

for word,n in voc.items():#для слова в словаре
        positions = range(-min(len_search+1, len(word) - 2), 0) # from -7 to 0
        for i in positions:
            cond_prob[(i, word[i])] += n
            total[i] += n # dictionary with prob of char words?

for posChar in cond_prob: #получаем из частот вероятности
        i = posChar[0]
        cond_prob[posChar] /= total[i]
pprint(cond_prob)

defaultdict(<function <lambda> at 0x00000139BF0CF9D8>,
            {(-8, 'а'): 0.09941203517927374,
             (-8, 'б'): 0.026301874856937378,
             (-8, 'в'): 0.0035505586808293483,
             (-8, 'г'): 0.013059337263788832,
             (-8, 'д'): 0.02273087150040268,
             (-8, 'е'): 0.01625632331771387,
             (-8, 'ж'): 0.0013509521005300783,
             (-8, 'з'): 0.0022160424960623957,
             (-8, 'и'): 0.05084149884327232,
             (-8, 'й'): 0.019474956675326924,
             (-8, 'к'): 0.0223849155173886,
             (-8, 'л'): 0.09523691169944563,
             (-8, 'м'): 0.02323517350113468,
             (-8, 'н'): 0.04672089449865847,
             (-8, 'о'): 0.0530928184268263,
             (-8, 'п'): 0.02111774649965104,
             (-8, 'р'): 0.09086696248642534,
             (-8, 'с'): 0.05837636403067984,
             (-8, 'т'): 0.06998292669140606,
             (-8, 'у'): 0.04513542994150818,
             (-8, 'ф'): 0.001564218129

In [24]:
# информанты - это буквы с макс значением КФ в каждой позиции
max_cond = defaultdict(lambda: 0.0)
maxlet = ['']*8
    #для каждой позиции ищем букву с наибольшим значением условной вероятности,
for posChar in cond_prob:#цикл по позициям букв в условной вероятности
        aff_len = posChar[0]
        if cond_prob[posChar] > max_cond[aff_len]:
            max_cond[aff_len] = cond_prob[posChar]
            maxlet[-aff_len] = posChar[1]

print("Наиболее частые буквы по позициям:\n============================\n", maxlet[-1:0:-1],"\n")

print("Максимальные вероятности по позициям:\n============================\n", max_cond,"\n")
    #порог медиального разбиения - половина условной вероятности , буквы с УВ не меньше порога - верхнее подмножеств
cond_prob_sup = {}
for posChar in cond_prob:
            i = posChar[0]
            if cond_prob[posChar] > THRESHOLD_COEFF * max_cond[i]:
                cond_prob_sup[posChar] = cond_prob[posChar]

Наиболее частые буквы по позициям:
 ['а', 'а', 'т', 'а', 'т', 'а', 'н'] 

Максимальные вероятности по позициям:
 defaultdict(<function <lambda> at 0x00000139C1CF1D08>, {-6: 0.13725284548556266, -5: 0.10984460793132571, -4: 0.13300193926114215, -3: 0.11746170593037474, -2: 0.21185955687204952, -1: 0.22671356984582827, -7: 0.10984867258690777}) 



In [25]:
# КФ = условная вер по данной позиции / безусл вероятность
cf = {}
for posChar in cond_prob_sup:
        char = posChar[1]
        cf[posChar] = cond_prob_sup[posChar] / prob[char]

print("КФ для верхних подмножества:\n====================\n");
for aff_len in set(map(itemgetter(0), cf.keys())):
        print(aff_len, "**")
        for k,v in cf.items():
            if k[0] == aff_len:
                print(k[1], "{:.4f}".format(v), end="  ")
        print("")

КФ для верхних подмножества:

-1 **
н 3.1559  р 2.0092  а 1.1595  
-7 **
ы 0.9773  р 1.1054  а 0.8338  л 1.7512  т 1.0604  и 0.9824  
-6 **
р 0.9975  а 1.0418  т 0.9701  э 1.0297  л 1.6040  
-5 **
а 0.7812  н 0.8144  т 1.4718  и 0.8949  р 1.1912  л 1.9490  
-4 **
а 1.0095  ы 1.4735  т 1.2438  и 1.2629  э 1.0359  л 1.7136  
-3 **
н 1.4117  л 1.3875  т 1.5738  а 0.8120  р 1.3286  
-2 **
а 1.6081  э 1.7332  


In [26]:
informants = []
for aff_len in range(-len_search, 0):
        kmax = max({k for k in cf if k[0] == aff_len}, key=lambda k: cf[k])
        informants.append((kmax[1], aff_len, cf[kmax]))

informants.sort(key = itemgetter(2), reverse=True)
informants

[('н', -1, 3.15594279017803), ('л', -5, 1.94903926924671), ('л', -7, 1.751239386475054), ('э', -2, 1.7331937693949762), ('л', -4, 1.7135874901772652), ('л', -6, 1.604008680102341), ('т', -3, 1.573810412515745)]

In [29]:
#подсчитываем частоты букв и строим дерево оконочаний
prob = defaultdict(lambda: 0)
trie = {'n':0}
for w,n in voc.items(): #для каждого слова в списке
        word = w[::-1]  # переворачиваем слово, читаем слово с конца
        current_dict = trie
        trie['n'] += n
        for letter in word:  # для буквы в слове
            prob[letter]+=n
            current_dict = current_dict.setdefault(letter, {'n': 0}) #получить значение из словаря по ключу.
                                                                     #Автоматически добавляет элемент словаря, если он отсутствует.
            current_dict['n']+=n
        current_dict['#'] = n

total = sum([n for n in prob.values()])#84263863
for k,v in prob.items():
    prob[k] = v/total

In [30]:
def extend_right(voc, char, pos, cf):
    if pos == -1:#если информант в последней позиции, то расширять некуда
        return char #возвращаем информант как аффикс
    d = defaultdict(int)
    for w,n in voc.items():#для буквы и частоты в словаре
        if w[pos:pos+1]==char: #если буква в позиции равна нашей, то посчитаем это окончание
            d[w[pos+1:]]+=n
    return char+max(d.keys(), key=lambda end: d[end]) #прибавляем к информанту самое частое окончание

In [31]:
for pos in range (-7,0):
    print(extend_right(voc, char, pos,cf))

ларыгар
лларын
ларын
ллар
лар
ла
л


In [32]:
def extend_left(affix, trie, len_search):
    #расширяем аффикс влево используя trie
   # import ipdb; ipdb.set_trace()
    current_dict = trie
    for ch in affix[::-1]:
        current_dict = current_dict[ch]
    aff_len = len(affix)

In [33]:
informants[0][0]

'н'

In [41]:
print(extend_left(informants[0][0],trie,len_search))

None


### ищем перепады перед потенциальными аффиксами

In [142]:
pprint(trie['р']['а']['г']['ы']['р']['а']['л']['n']) #лар-ы-гар

23661


In [141]:
pprint(trie['р']['а']['г']['ы']['р']['а']['л']['а']['n']) #а-лар-ы-гар

6653


In [187]:
trie['р']['а']['г']['ы']['р']['а']['л']['а']['n']/trie['р']['а']['г']['ы']['р']['а']['л']['n']

0.28118000084527284

In [189]:
pprint(trie['р']['а']['г']['ы']['р']['а']['л']['ы']['n']/trie['р']['а']['г']['ы']['р']['а']['л']['n'])

0.21727737627319219


In [185]:
pprint(trie['р']['а']['г']['ы']['р']['а']['л']['л']['n']/trie['р']['а']['г']['ы']['р']['а']['л']['n'])

0.31237056760069315


In [184]:
pprint(trie['р']['а']['г']['ы']['р']['а']['л']['у']['n']/trie['р']['а']['г']['ы']['р']['а']['л']['n'])

0.10688474705211107


In [183]:
pprint(trie['р']['а']['г']['ы']['р']['а']['л']['о']['n']/trie['р']['а']['г']['ы']['р']['а']['л']['n'])

0.05249144161278053


In [139]:
pprint(trie['р']['а']['г']['ы']['р']['а']['л']['л']['n'])

7391


In [160]:
pprint(trie['р']['а']['г']['ы']['р']['а']['л']['у']['n'])

2529


In [171]:
for letter in ['абвгдеёжзийклмнопрстуфхчцшщэюя']:
    print(letter)
    tr = trie['р']['а']['г']['ы']['р']['а']['л']['у']['n']
tr

абвгдеёжзийклмнопрстуфхчцшщэюя


2529

In [47]:
pprint(trie['р']['а']['г']['ы']['р']['а']['n'])

48372


In [123]:
from pprint import pprint
pprint(trie['р']['а']['з']['а']['х']['n'])

3


In [194]:
str = "ларыгар"

'л'

In [134]:
pprint(trie['р']['а']['г']['ы']['р']['а']['л']['ы'])

{'#': 253,
 'n': 5141,
 'г': {'n': 32,
       'й': {'n': 6, 'а': {'n': 6, 'м': {'#': 6, 'n': 6}}},
       'л': {'n': 24,
             'ы': {'n': 24,
                   'й': {'n': 2, 'а': {'#': 2, 'n': 2}},
                   'с': {'#': 22, 'n': 22}}},
       'р': {'n': 1,
             'ы': {'n': 1,
                   'л': {'n': 1, 'ы': {'n': 1, 'б': {'#': 1, 'n': 1}}}}},
       'ы': {'n': 1, 'р': {'n': 1, 'а': {'#': 1, 'n': 1}}}},
 'д': {'n': 1213,
       'р': {'n': 1213,
             'а': {'#': 1180,
                   'n': 1213,
                   'к': {'n': 33, 'к': {'n': 33, 'ы': {'#': 33, 'n': 33}}}}}},
 'й': {'n': 18,
       'а': {'n': 18, 'а': {'n': 18, 'г': {'n': 18, 'у': {'#': 18, 'n': 18}}}}},
 'к': {'n': 7,
       'а': {'n': 3,
             'а': {'n': 2,
                   'п': {'n': 2,
                         'п': {'n': 2, 'а': {'n': 2, 'л': {'#': 2, 'n': 2}}}}},
             'т': {'n': 1,
                   'с': {'n': 1, 'а': {'n': 1, 'б': {'#': 1, 'n': 1}}}}},
       'с'

                         'ы': {'n': 1,
                               'й': {'n': 1,
                                     'а': {'n': 1, 'н': {'#': 1, 'n': 1}}}}},
                   'н': {'n': 4,
                         'м': {'n': 2,
                               'у': {'n': 2,
                                     'р': {'n': 1,
                                           'о': {'n': 1,
                                                 'т': {'#': 1, 'n': 1}}},
                                     'т': {'n': 1, 'у': {'#': 1, 'n': 1}}}},
                         'н': {'n': 2,
                               'а': {'n': 1,
                                     'а': {'n': 1,
                                           'р': {'n': 1,
                                                 'а': {'n': 1,
                                                       'б': {'#': 1,
                                                             'n': 1}}}}},
                               'ы': {'n': 1,
                   

 'ҕ': {'n': 7,
       'а': {'n': 7,
             'а': {'n': 7,
                   'м': {'n': 5, 'у': {'n': 5, 'к': {'#': 5, 'n': 5}}},
                   'н': {'n': 1,
                         'н': {'n': 1, 'о': {'n': 1, 'у': {'#': 1, 'n': 1}}}},
                   'т': {'n': 1,
                         'х': {'n': 1,
                               'а': {'n': 1,
                                     'а': {'n': 1,
                                           'р': {'n': 1,
                                                 'ы': {'#': 1, 'n': 1}}}}}}}}},
 'ҥ': {'n': 4,
       'а': {'n': 2,
             'а': {'n': 1, 'ь': {'n': 1, 'д': {'#': 1, 'n': 1}}},
             'д': {'n': 1,
                   'а': {'n': 1, 'ь': {'n': 1, 'д': {'#': 1, 'n': 1}}}}},
       'ҥ': {'n': 2,
             'ы': {'n': 2,
                   'с': {'n': 2,
                         'р': {'n': 2,
                               'а': {'n': 2, 'с': {'#': 2, 'n': 2}}}}}}},
 'һ': {'n': 7,
       'а': {'n': 6,
             'а

In [76]:
pprint(trie['н']['а']['т']['n'])

155812
